Part of the analysis consist of wrangling with data till will get the final form.
This notebook has the functions which go in the database and aggregate needed columns

These are the columns and a little explanation for each one

**ID** - ego unique identification
**EGO_COUNTRY** - country of current affiliation of ego
**EGO_PAPERS** - total number of ego publications *(to be noted, data was scraped in september 2018 - january 2019, Scopus keep indexing documents, from later and former years)*
**EGO_CITATIONS** - total number of ego citations, *same note as above*
**EGO_COAUTHORS** - total number of authors in ego network
**EGO_EDGES** - unique collaboration with each author in each year, if ego did wrote in 2010 and 2011 with same author 5 documents, will be counted as 2, (2010 and 2011)
**ALTER_DOMESTIC** - how many authors have same affiliation country as ego
**ALTER_NONDOMESTIC** - how many authors does not have same affiliation country as ego
**ALTER_CITATIONS** - cumulated sum of coauthors citations
**ALTER_PAPERS** - cumulated sum of coauthors articles (with or without ego)
**ALTER_EGO_PAPERS** - how many articles did ego wrote with his coauthors
**ALTER_EGO_CITATIONS** - to be excluded (not all articles does have citations)
**ALTER_COUNTRY** - number of different nondomestic countries
**ALTER_COUNTRIES** - list of nondomestic countries
**ALTER_MAX_PAPERS** - the author from ego network which has most papers
**ALTER_MAX_CITATIONS** - the author from ego network which has most citations
**EGO_BETWEENES** - centrality score of ego in his network


In [1]:
%config InlineBackend.figure_format = 'retina'
import findspark
findspark.init()

import ast
import pandas as pd
import numpy as np
import os
from pyspark import SparkContext,SparkConf
from pyspark.sql import SQLContext, Row, SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F
import time
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.style as style
from slugify import slugify

In [2]:
sc = SparkContext.getOrCreate()
sqlContext = SQLContext(sc)

Load data from parquet files, which were saved from the sqlite database using `to_parquet.py` file

In [19]:
authors = sqlContext.read.parquet('./data/authors')
collaborations = sqlContext.read.parquet('./data/collaborations')
collaborations = collaborations.withColumn('published', F.to_date(F.col('published')))
collaborations = collaborations.withColumn('year', F.year(F.col('published')))
collaborations = collaborations.withColumnRenamed('author_id', 'auth_id')
collaborations = collaborations.withColumn('auth_id', collaborations.auth_id.cast('bigint'))
ego_alters = sqlContext.read.parquet('./data/ego_alters')
# Compute authors citations based on downloaded papers
# Because not all articles for authors were scraped we need to base our analysis on grounded data, not missing
authors = authors.join(collaborations, collaborations.auth_id == authors.id, 'INNER') \
        .groupby([authors.id]) \
        .agg( \
            F.count('abs_id').alias('EGO_PAPERS'), \
            F.first('cited_by_count').alias('CITATIONS_SCOPUS'), \
            F.sum('cited_by').alias('EGO_CITATIONS'), \
            F.first('cat').alias('cat'), \
            F.first('country').alias('country'))
# Exclude articles which have errors
collaborations = collaborations.filter(collaborations.abs_id != 85032509284)

Search for most citated authors on PHYS category from UE and get the EGOs

In [28]:
countries = ['Austria', 'Belgium', 'Bulgaria', 'Croatia', 'Cyprus', 'Czech Republic', 
             'Denmark', 'Estonia', 'Finland', 'France', 'Germany', 'Greece', 'Hungary', 
             'Ireland', 'Italy', 'Latvia', 'Lithuania', 'Luxembourg', 'Malta', 'Netherlands',
             'Poland', 'Portugal', 'Romania', 'Slovakia', 'Slovenia', 'Spain', 'Sweden', 'United Kingdom']

most_cited_phys = authors.orderBy(F.col('EGO_CITATIONS').desc()) \
    .filter(F.col('country').isin(countries)) \
    .filter(F.col('cat').like('%PHYS%')) \
    .limit(1200) \
    .select(authors.id.alias('ID'), authors.country.alias('EGO_COUNTRY'), 'EGO_PAPERS', 'EGO_CITATIONS')

Get the coauthors of each ego
- First get ego collaborations
- Then get authors of collaborations and exclude duplicate articles of ego
- Obtain data about coauthors

In [35]:
# Select each ego articles 
most_cited_phys_coauthors = most_cited_phys.join(collaborations, F.col('ID') == collaborations.auth_id.alias('todel1'), 'inner')
most_cited_phys_coauthors = most_cited_phys_coauthors.select('ID', 'EGO_COUNTRY', 'EGO_PAPERS', 'EGO_CITATIONS', F.col('abs_id').alias('coll_id'))

# Based on ego articles select his coauthors
most_cited_phys_coauthors = most_cited_phys_coauthors \
    .join(collaborations, most_cited_phys_coauthors.coll_id == collaborations.abs_id, 'INNER') \
    .filter(F.col('ID') != F.col('auth_id'))

# And keep only columns which interest us
most_cited_phys_coauthors = most_cited_phys_coauthors.select('ID', 'EGO_COUNTRY', 'EGO_PAPERS', 'EGO_CITATIONS', F.col('auth_id').alias('coauthor_id'))

most_cited_phys_coauthors = most_cited_phys_coauthors.join(authors, most_cited_phys_coauthors.coauthor_id == authors['ID']) \
    .select( \
        most_cited_phys_coauthors['ID'], \
        most_cited_phys_coauthors['EGO_COUNTRY'], \
        most_cited_phys_coauthors['EGO_PAPERS'], \
        most_cited_phys_coauthors['EGO_CITATIONS'], \
        'coauthor_id', \
        F.col('country').alias('coauthor_country'), \
        authors['EGO_CITATIONS'].alias('coauthor_citations'), \
        authors['EGO_PAPERS'].alias('coauthor_papers'))

most_cited_phys_coauthors.show(10)


+-----------+-----------+----------+-------------+-----------+----------------+------------------+---------------+
|         ID|EGO_COUNTRY|EGO_PAPERS|EGO_CITATIONS|coauthor_id|coauthor_country|coauthor_citations|coauthor_papers|
+-----------+-----------+----------+-------------+-----------+----------------+------------------+---------------+
| 6506585188|Netherlands|      1231|        22983| 6504152908|     Netherlands|             22983|           1231|
| 6506585188|Netherlands|      1231|        22983| 6504152908|     Netherlands|             22983|           1231|
| 6506585188|Netherlands|      1231|        22983| 6504152908|     Netherlands|             22983|           1231|
|36048731400|Netherlands|      1028|        42326| 6504152908|     Netherlands|             42326|           1028|
| 6603547661|Netherlands|       699|        12809| 6504152908|     Netherlands|             12809|            699|
| 6603547661|Netherlands|       699|        12809| 6504152908|     Netherlands| 

Code down from here should be organized and explain, feel free to test it or add comments
This function does build network of each ego and saves it

In [6]:
def ego_alters():
    publications = collaborations.filter(collaborations.auth_id.isin(list(networks.index))).select(collaborations.auth_id.alias('ego_id'),collaborations.abs_id.alias('todel1'))
    alter_publications = publications.join(collaborations, publications.todel1 == collaborations.abs_id, 'inner')
    ego_alters = alter_publications.dropDuplicates(['ego_id', 'auth_id']).groupby([alter_publications.ego_id, alter_publications.auth_id]).count() 
    # alter_publications.show(20)
    # ego_alters.write.parquet('./data/ego_alters')

Look how much missing data there is in collaboration papers

In [5]:
authors_cits = authors.join(collaborations, collaborations.auth_id == authors.id, 'inner') \
    .groupby([authors.id]) \
    .agg(F.sum(collaborations.cited_by).alias('cited_by_count2'))

#authors_cits.show(20)
collaborations.select([F.count(F.when(F.isnan(c) | F.col(c).isNull(), c)).alias(c) for c in collaborations.columns if c != 'published']).show()


+------+--------+--------+-------+----+
|abs_id|cited_by|keywords|auth_id|year|
+------+--------+--------+-------+----+
|     0|     162|12900432|      0| 280|
+------+--------+--------+-------+----+



In [7]:
def ego_papers():
    papers = authors.filter(authors.id.isin(list(networks.index))).join(collaborations, collaborations.auth_id == authors.id.alias('ID'), 'left') \
        .groupby([authors.id.alias('ID')]).agg(F.count('abs_id').alias('EGO_PAPERS2'))
    to_p = papers.toPandas()
    to_p['ID'] = to_p['ID'].astype(str)
    return to_p

Get authors with missing countries

In [4]:
alters_data = ego_alters.join(authors, ego_alters.auth_id == authors.id, 'left')
missing_country = alters_data.filter(alters_data.country.isNull()).groupby(alters_data.auth_id).count().toPandas()
missing_country.to_csv('./missing_country_phys.csv')

In [8]:
def grown_percent(row):
        if row['EGO_PAPERS2'] > 0:
            diff = row['EGO_PAPERS'] - row['EGO_PAPERS2']
            return (diff*100)/row['EGO_PAPERS2']
        else:
            return 100
def compute_percent_of_paper_grown():
    grouped = networks.combine_first(ego_papers().set_index('ID'))
    
    grouped['PAPERS_GROWN'] = grouped.apply(lambda row: grown_percent(row), axis=1)
    q_75 = grouped[grouped['PAPERS_GROWN'] <= grouped['PAPERS_GROWN'].quantile(.75)]
    return q_75['PAPERS_GROWN'].mean()

percent_of_paper_grown = compute_percent_of_paper_grown()

In [9]:
networks = networks.combine_first(ego_papers().set_index('ID'))
networks['PAPERS_GROWN'] = networks.apply(lambda row: grown_percent(row), axis=1)

def grow_papers(row):
    if row['PAPERS_GROWN'] >= percent_of_paper_grown:
        return round((row['EGO_PAPERS2']*percent_of_paper_grown)/100,0)
    else:
        return row['EGO_PAPERS2']

networks['EGO_PAPERS'] = networks.apply(lambda row: grow_papers(row), axis=1)
networks.drop(['EGO_PAPERS2', 'PAPERS_GROWN'], axis=1, inplace=True)

In [14]:
def alter_provenance():
    alters_country = ego_alters.join(authors, ego_alters.auth_id == authors.id).select(F.col('ego_id'), F.col('auth_id'), F.col('country').alias('alter_country'))
    alters_country = alters_country.join(authors, alters_country.ego_id == authors.id).select(F.col('ego_id'),F.col('country').alias('ego_country'), F.col('alter_country'), F.col('auth_id'))

    def check_provenance(ego_c, alter_c):
        if ego_c == alter_c:
            return 'domestic'
        else:
            return 'nondomestic'

    provenance = F.udf(check_provenance, StringType())

    alters_country = alters_country.withColumn('provenance', provenance('ego_country', 'alter_country'))

    alters_country_provenance = alters_country.groupby('ego_id').pivot('provenance', ['domestic', 'nondomestic']).count()
    alters_country_provenance = alters_country_provenance.withColumnRenamed('ego_id', 'ID')
    alters_country_provenance = alters_country_provenance.withColumnRenamed('domestic', 'ALTER_DOMESTIC')
    alters_country_provenance = alters_country_provenance.withColumnRenamed('nondomestic', 'ALTER_NONDOMESTIC')
    
    to_p = alters_country_provenance.toPandas()
    to_p['ID'] = to_p['ID'].astype(str)
    return to_p


def alter_countries():
    alters_country = ego_alters.join(authors, ego_alters.auth_id == authors.id).select(F.col('ego_id'), F.col('auth_id'), F.col('country').alias('alter_country'))
    alters_country = alters_country.join(authors, alters_country.ego_id == authors.id).select(F.col('ego_id'),F.col('country').alias('ego_country'), F.col('alter_country'), F.col('auth_id'))

    countries = alters_country.groupby('ego_id').agg(F.first('ego_country').alias('ego_country'),F.collect_set('alter_country').alias('ALTER_COUNTRIES'))
    countries = countries.withColumnRenamed('ego_id', 'ID')
    
    to_p = countries.toPandas()
    to_p['ID'] = to_p['ID'].astype(str)
    
    def map_alter_countries(row):
        l = list(row['ALTER_COUNTRIES'])
        l = [li for li in l if li != row['ego_country']]
        return ','.join(l)
    
    def count_alter_countries(row):
        l = list(row['ALTER_COUNTRIES'])
        l = [li for li in l if li != row['ego_country']]
        return len(l)
        
    to_p['ALTER_COUNTRIES'] = to_p.apply(lambda row: map_alter_countries(row), axis=1)
    to_p['ALTER_COUNTRY'] = to_p.apply(lambda row: count_alter_countries(row), axis=1)
    to_p.drop(['ego_country'], axis=1, inplace=True)
    return to_p


In [15]:
networks = networks.combine_first(alter_provenance().set_index('ID'))
networks = networks.combine_first(alter_countries().set_index('ID'))
networks.to_excel('./baza_date_fizica_temp.xlsx')